<a href="https://colab.research.google.com/github/313ctr0nz/devz_workbook/blob/main/313avax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
api_key = ""
wallet_address = "0x5e8dcda987e97f78baf533bde8493a0a726ad1ef" 
contract_address = "0x95189f25b4609120F72783E883640216E92732DA" # Thor

In [8]:
# MOVE ALL OF THIS TO A MODULE PLEASE!!!!

import argparse
import json
import requests
import time 
import datetime 

signatures = {
    "Transfer"  : "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef",
    "Approve"   : "0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925",
    "Sync"      : "0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1",
    "Swap"      : "0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822",
}

addrs = {}

def fetchtkn(contractAddr, apikey):
    url = f"https://api.snowtrace.io/api?module=account&action=tokentx&contractaddress={contractAddr}&page=1&offset=1&sort=desc&apikey={apikey}"
    # print(url)
    try:
        res = requests.get(url, timeout=1)
        r=res.json().get("result")[0]
        # print(r)
        return { "name" : r.get('tokenName'), "symbol" : r.get('tokenSymbol'), "decimal" : r.get('tokenDecimal')}
    except:
        print("Timeout")
        return {}

def matchcontractinlog(contract_address, log):
    for topic in log.get("topics"):
        # print(topic)
        if contract_address[2:] in topic:
            return True 
    return False

def parsedata(log):
    print(addrs[log.get("address")])
    data = []
    if len(log.get("data")) == 66:
        data.append(log.get("data"))
    if len(log.get("data")) == 130:
        data.append(log.get("data")[0:66])
        data.append(log.get("data")[66:])
        # print(data)
    if len(log.get("data")) == 258:
        data.append(log.get("data")[0:66])
        data.append(log.get("data")[66:])
        # print(data)

    for d in data:
        print(int(d,16)/(10**int(addrs[log.get("address")].get("decimal"))))

def fetchdate(blockno, apikey):
    url = f"https://api.snowtrace.io/api?module=block&action=getblockreward&blockno={blockno}&apikey={apikey}"
    # print(url)
    res = requests.get(url)
    # print(json.dumps(res.json()))   
    return datetime.datetime.fromtimestamp(int(res.json().get("result").get("timeStamp"))).strftime("%d-%m-%Y %H:%M:%S")

def fetchtxn(r, apikey, contract_address=None):
    # print(r)
    url = "https://api.snowtrace.io/api?module=proxy&action=eth_getTransactionReceipt&txhash="+r.get("hash")+"&apikey=" + apikey
    # print(url)
    res = requests.get(url)
    # print(json.dumps(res.json()))   
    return res.json()

def process(txn, apikey):
    if not txn.get("result").get("logs"):
        return 

    for log in txn.get("result").get("logs"):
        if signatures.get("Transfer") not in log.get("topics"):
            continue
        if not matchcontractinlog(contract_address.lower(), log):
            continue

        if not addrs.get(log.get("address")):
            time.sleep(0.2)
            token = fetchtkn(log.get("address"),apikey)
            # print(token)
            if "symbol" not in token:
                print("token lookup error")
                continue
        else:
            token = addrs.get(log.get("address"))

        txndate = fetchdate(int(txn.get("result").get("blockNumber"),16),apikey)
        print(txndate)
    
        addrs[log.get("address")] = token
        parsedata(log)

    print("-"*50)

def fetch(address, apikey):
    url = "https://api.snowtrace.io/api?module=account&action=txlist&contractaddress="+contract_address+"&address="+address+"&sort=asc&apikey=" + apikey
    res = requests.get(url)
    # print(res.json())

    for r in res.json().get("result"):
        time.sleep(0.2)
        txn = fetchtxn(r, apikey, contract_address)

        # print(txn)
        process(txn, apikey)




In [ ]:
fetch(wallet_address, api_key)